# 🎬 Video Trimmer - Google Colab Edition\n\n<a href=\"https://colab.research.google.com/github/nipunbatra/video-toolkit/blob/main/video_trimmer_colab.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>\n\nClean, simple video trimming tool with Google Drive integration.\n\n**Features:**\n- ✂️ Precise video trimming with visual sliders\n- 🎵 Audio extraction (AAC format)\n- 📁 Local files + Google Drive integration\n- 🚀 Fast processing using ffmpeg\n\n**Supported formats:** MP4, MOV, AVI, MKV"

## 🚀 Setup and Installation

In [ ]:
# Install required packages
!pip install google-api-python-client google-auth-oauthlib google-auth-httplib2

# Install ffmpeg for video processing
!apt update &> /dev/null
!apt install ffmpeg &> /dev/null

print("✅ Setup complete!")

In [ ]:
# Import libraries
import os
import tempfile
import subprocess
import logging
import re
import json
import pickle
from pathlib import Path
from IPython.display import display, HTML, Video, Audio, clear_output
import ipywidgets as widgets
from google.colab import drive, files
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import io

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("📚 Libraries imported successfully!")

## 🔧 Core Functions

In [ ]:
# Google Drive configuration
SCOPES = ['https://www.googleapis.com/auth/drive.file']
TOKEN_FILE = 'oauth_token.pickle'
CREDENTIALS_FILE = 'oauth_credentials.json'

def get_google_drive_service():
    """Get Google Drive service with OAuth credentials"""
    creds = None
    
    # Check if token file exists
    if os.path.exists(TOKEN_FILE):
        with open(TOKEN_FILE, 'rb') as token:
            creds = pickle.load(token)
    
    # If no valid credentials, authenticate
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            if not os.path.exists(CREDENTIALS_FILE):
                print("⚠️ Upload your oauth_credentials.json file first!")
                return None
            
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        
        # Save credentials for next run
        with open(TOKEN_FILE, 'wb') as token:
            pickle.dump(creds, token)
    
    return build('drive', 'v3', credentials=creds)

def extract_drive_file_id(input_str):
    """Extract file ID from Drive link or return input if it's already a file ID"""
    if not input_str:
        return None
    
    input_str = input_str.strip()
    
    # Drive link patterns
    patterns = [
        r'https://drive\.google\.com/file/d/([a-zA-Z0-9-_]+)',
        r'drive\.google\.com/file/d/([a-zA-Z0-9-_]+)',
        r'/d/([a-zA-Z0-9-_]+)/',
        r'id=([a-zA-Z0-9-_]+)'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, input_str)
        if match:
            return match.group(1)
    
    # Check if it's already a file ID
    if re.match(r'^[a-zA-Z0-9-_]{25,}$', input_str):
        return input_str
    
    return None

def extract_drive_folder_id(input_str):
    """Extract folder ID from Drive folder link"""
    if not input_str:
        return None
    
    input_str = input_str.strip()
    
    # Folder link patterns
    patterns = [
        r'https://drive\.google\.com/drive/folders/([a-zA-Z0-9-_]+)',
        r'drive\.google\.com/drive/folders/([a-zA-Z0-9-_]+)',
        r'/folders/([a-zA-Z0-9-_]+)',
        r'folders/([a-zA-Z0-9-_]+)'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, input_str)
        if match:
            return match.group(1)
    
    # Check if it's already a folder ID
    if re.match(r'^[a-zA-Z0-9-_]{25,}$', input_str):
        return input_str
    
    return None

def download_from_drive(service, file_id, filename):
    """Download a file from Google Drive"""
    try:
        request = service.files().get_media(fileId=file_id)
        temp_dir = tempfile.mkdtemp()
        temp_file = os.path.join(temp_dir, filename)
        
        with open(temp_file, 'wb') as fh:
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                if status:
                    print(f"📥 Download: {int(status.progress() * 100)}%")
        
        print(f"✅ Downloaded: {filename}")
        return temp_file
    except Exception as e:
        print(f"❌ Download error: {e}")
        return None

def get_video_duration(video_file):
    """Get video duration in seconds"""
    try:
        cmd = ["ffprobe", "-v", "quiet", "-print_format", "json", "-show_format", video_file]
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            data = json.loads(result.stdout)
            return float(data['format']['duration'])
        return 0
    except:
        return 0

def format_time(seconds):
    """Format seconds to mm:ss"""
    if seconds is None:
        return "0:00"
    minutes = int(seconds // 60)
    secs = int(seconds % 60)
    return f"{minutes}:{secs:02d}"

def process_video_trim(video_file, start_time, end_time):
    """Process video trimming using ffmpeg"""
    if not video_file or start_time is None or end_time is None:
        return None, None, "❌ Missing video file or times"
    
    try:
        start_seconds = float(start_time)
        end_seconds = float(end_time)
        
        if start_seconds >= end_seconds:
            return None, None, "❌ Start time must be less than end time"
        
        # Create output files
        temp_dir = tempfile.mkdtemp()
        base_name = Path(video_file).stem
        output_video = os.path.join(temp_dir, f"{base_name}_trimmed.mp4")
        output_audio = os.path.join(temp_dir, f"{base_name}_trimmed.aac")
        
        # Convert seconds to HH:MM:SS format
        def seconds_to_time(seconds):
            hours = int(seconds // 3600)
            minutes = int((seconds % 3600) // 60)
            secs = seconds % 60
            return f"{hours:02d}:{minutes:02d}:{secs:06.3f}"
        
        start_time_str = seconds_to_time(start_seconds)
        end_time_str = seconds_to_time(end_seconds)
        duration = end_seconds - start_seconds
        
        print(f"🎬 Trimming from {format_time(start_seconds)} to {format_time(end_seconds)}...")
        
        # Trim video
        video_cmd = [
            "ffmpeg", "-i", video_file,
            "-ss", start_time_str,
            "-t", str(duration),
            "-c", "copy",
            "-avoid_negative_ts", "make_zero",
            output_video, "-y"
        ]
        
        # Extract audio
        audio_cmd = [
            "ffmpeg", "-i", video_file,
            "-ss", start_time_str,
            "-t", str(duration),
            "-vn", "-acodec", "aac",
            output_audio, "-y"
        ]
        
        # Run commands
        result1 = subprocess.run(video_cmd, capture_output=True, text=True)
        result2 = subprocess.run(audio_cmd, capture_output=True, text=True)
        
        if result1.returncode == 0 and result2.returncode == 0:
            return output_video, output_audio, f"✅ Trimmed successfully! Duration: {format_time(duration)}"
        else:
            return None, None, f"❌ FFmpeg error: {result1.stderr or result2.stderr}"
            
    except Exception as e:
        return None, None, f"❌ Error: {str(e)}"

print("🔧 Core functions loaded!")

## 📁 Input Source Selection

In [ ]:
# Create input widgets
input_method = widgets.ToggleButtons(
    options=['📁 Upload File', '☁️ Google Drive'],
    description='Input:',
    button_style='info'
)

upload_widget = widgets.FileUpload(
    accept='.mp4,.mov,.avi,.mkv',
    multiple=False,
    description='Upload Video'
)

drive_input = widgets.Text(
    placeholder='Enter Drive file ID or link',
    description='Drive File:',
    style={'description_width': 'initial'}
)

load_button = widgets.Button(
    description='📥 Load Video',
    button_style='primary'
)

status_output = widgets.Output()

# Show/hide inputs based on method
def on_input_method_change(change):
    if change['new'] == '📁 Upload File':
        upload_widget.layout.display = 'block'
        drive_input.layout.display = 'none'
    else:
        upload_widget.layout.display = 'none'
        drive_input.layout.display = 'block'

input_method.observe(on_input_method_change, names='value')

# Initial state
drive_input.layout.display = 'none'

display(widgets.VBox([
    widgets.HTML("<h3>📂 Select Input Source</h3>"),
    input_method,
    upload_widget,
    drive_input,
    load_button,
    status_output
]))

print("📂 Input widgets ready!")

## 🎬 Video Player & Trimmer

In [ ]:
# Video player and controls
video_player = widgets.Output()
video_info = widgets.HTML(value="<p>Load a video to see information</p>")

# Trim range sliders
start_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=100,
    step=0.1,
    description='▶️ Start:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)

end_slider = widgets.FloatSlider(
    value=100,
    min=0,
    max=100,
    step=0.1,
    description='⏹️ End:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)

start_time_display = widgets.HTML(value="<b>Start:</b> 0:00")
end_time_display = widgets.HTML(value="<b>End:</b> 0:00")

trim_button = widgets.Button(
    description='✂️ Trim Video',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

trim_status = widgets.Output()

# Update time displays
def update_start_display(change):
    start_time_display.value = f"<b>Start:</b> {format_time(change['new'])}"
    # Ensure end >= start
    if end_slider.value < change['new']:
        end_slider.value = change['new']

def update_end_display(change):
    end_time_display.value = f"<b>End:</b> {format_time(change['new'])}"
    # Ensure end >= start
    if change['new'] < start_slider.value:
        end_slider.value = start_slider.value

start_slider.observe(update_start_display, names='value')
end_slider.observe(update_end_display, names='value')

display(widgets.VBox([
    widgets.HTML("<h3>🎬 Video Player & Trimmer</h3>"),
    video_player,
    video_info,
    widgets.HTML("<h4>✂️ Trim Range Selector</h4>"),
    start_slider,
    end_slider,
    widgets.HBox([start_time_display, end_time_display]),
    trim_button,
    trim_status
]))

print("🎬 Video player widgets ready!")

## 💾 Output & Save Options

In [ ]:
# Output widgets
output_video_player = widgets.Output()
output_audio_player = widgets.Output()

# Save options
drive_folder_input = widgets.Text(
    placeholder='Drive folder ID or link (optional)',
    description='Drive Folder:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

save_drive_button = widgets.Button(
    description='☁️ Save to Drive',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

download_button = widgets.Button(
    description='💾 Download Files',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

save_status = widgets.Output()

display(widgets.VBox([
    widgets.HTML("<h3>💾 Output & Save</h3>"),
    widgets.HTML("<h4>📤 Output Files</h4>"),
    output_video_player,
    output_audio_player,
    widgets.HTML("<h4>🔄 Save Options</h4>"),
    drive_folder_input,
    widgets.HBox([save_drive_button, download_button]),
    save_status
]))

print("💾 Output widgets ready!")

## 🔗 Event Handlers

In [ ]:
# Global variables
current_video_file = None
current_output_video = None
current_output_audio = None

def load_video(button):
    global current_video_file
    
    with status_output:
        clear_output()
        
        try:
            if input_method.value == '📁 Upload File':
                if not upload_widget.value:
                    print("❌ Please select a video file")
                    return
                
                # Save uploaded file
                uploaded_file = list(upload_widget.value.values())[0]
                temp_dir = tempfile.mkdtemp()
                current_video_file = os.path.join(temp_dir, uploaded_file['metadata']['name'])
                
                with open(current_video_file, 'wb') as f:
                    f.write(uploaded_file['content'])
                
                print(f"✅ Uploaded: {uploaded_file['metadata']['name']}")
                
            else:  # Google Drive
                if not drive_input.value:
                    print("❌ Please enter a Drive file ID or link")
                    return
                
                file_id = extract_drive_file_id(drive_input.value)
                if not file_id:
                    print("❌ Invalid Drive file ID or link")
                    return
                
                # Get Drive service
                service = get_google_drive_service()
                if not service:
                    print("❌ Could not connect to Google Drive")
                    return
                
                # Get file info and download
                file_info = service.files().get(fileId=file_id).execute()
                filename = file_info['name']
                
                current_video_file = download_from_drive(service, file_id, filename)
                if not current_video_file:
                    print("❌ Failed to download from Drive")
                    return
            
            # Get video info and setup sliders
            duration = get_video_duration(current_video_file)
            if duration > 0:
                start_slider.max = duration
                end_slider.max = duration
                start_slider.value = 0
                end_slider.value = duration
                
                # Update video info
                video_info.value = f"<p><b>📹 Video loaded!</b><br>Duration: {format_time(duration)} ({duration:.1f}s)</p>"
                
                # Display video
                with video_player:
                    clear_output()
                    display(Video(current_video_file, width=600, height=400))
                
                print(f"🎬 Video ready for trimming!")
            else:
                print("⚠️ Could not determine video duration")
                
        except Exception as e:
            print(f"❌ Error loading video: {e}")

def trim_video(button):
    global current_output_video, current_output_audio
    
    with trim_status:
        clear_output()
        
        if not current_video_file:
            print("❌ Please load a video first")
            return
        
        # Process trimming
        output_video, output_audio, message = process_video_trim(
            current_video_file, start_slider.value, end_slider.value
        )
        
        print(message)
        
        if output_video and output_audio:
            current_output_video = output_video
            current_output_audio = output_audio
            
            # Display output video
            with output_video_player:
                clear_output()
                display(widgets.HTML("<h5>🎬 Trimmed Video</h5>"))
                display(Video(output_video, width=500, height=300))
            
            # Display output audio
            with output_audio_player:
                clear_output()
                display(widgets.HTML("<h5>🎵 Extracted Audio</h5>"))
                display(Audio(output_audio))

def save_to_drive(button):
    with save_status:
        clear_output()
        
        if not current_output_video or not current_output_audio:
            print("❌ Please trim a video first")
            return
        
        try:
            service = get_google_drive_service()
            if not service:
                print("❌ Could not connect to Google Drive")
                return
            
            # Get folder ID if specified
            folder_id = None
            if drive_folder_input.value:
                folder_id = extract_drive_folder_id(drive_folder_input.value)
            
            # Upload video
            video_name = os.path.basename(current_output_video)
            video_metadata = {'name': video_name}
            if folder_id:
                video_metadata['parents'] = [folder_id]
            
            video_media = MediaFileUpload(current_output_video, resumable=True)
            video_upload = service.files().create(
                body=video_metadata,
                media_body=video_media,
                fields='id,name'
            ).execute()
            
            # Upload audio
            audio_name = os.path.basename(current_output_audio)
            audio_metadata = {'name': audio_name}
            if folder_id:
                audio_metadata['parents'] = [folder_id]
            
            audio_media = MediaFileUpload(current_output_audio, resumable=True)
            audio_upload = service.files().create(
                body=audio_metadata,
                media_body=audio_media,
                fields='id,name'
            ).execute()
            
            location = f" to folder {folder_id}" if folder_id else " to Drive root"
            print(f"✅ Uploaded{location}:")
            print(f"📹 {video_upload['name']} (ID: {video_upload['id']})")
            print(f"🎵 {audio_upload['name']} (ID: {audio_upload['id']})")
            
        except Exception as e:
            print(f"❌ Upload failed: {e}")

def download_files(button):
    with save_status:
        clear_output()
        
        if not current_output_video or not current_output_audio:
            print("❌ Please trim a video first")
            return
        
        try:
            # Download video
            video_name = os.path.basename(current_output_video)
            files.download(current_output_video)
            
            # Download audio
            audio_name = os.path.basename(current_output_audio)
            files.download(current_output_audio)
            
            print(f"💾 Downloaded: {video_name} and {audio_name}")
            
        except Exception as e:
            print(f"❌ Download failed: {e}")

# Connect event handlers
load_button.on_click(load_video)
trim_button.on_click(trim_video)
save_drive_button.on_click(save_to_drive)
download_button.on_click(download_files)

print("🔗 Event handlers connected!")
print("\n🎉 Video Trimmer is ready to use!")
print("\n📋 Instructions:")
print("1. Choose input method and load a video")
print("2. Use the sliders to set trim start/end points")
print("3. Click 'Trim Video' to process")
print("4. Save to Drive or download the results")